## Price an American / European lookback put with the binomial tree model
### Implement the binomial tree model to price both European and American lookback puts. 

### 1. binomial tree model and method in Cheuk and Vorst (1997)
### 2. Monte Carlo
### 3. binomial tree model 
=========================

## binomial tree model and method in Cheuk and Vorst (1997)

In [3]:
# Price an American / European lookback put with the binomial tree model and method in Cheuk and Vorst (1997)
import numpy as np

# Input Info
s_p = 50
q = 0.0
sd1 = 0.4
r = 0.1
t = 0.25
period =1000
smaxt = 50
delt =  t / period
u =  np.exp(sd1 * (delt**0.5))   
d =  1/u        
v = smaxt / (d*s_p)
# pre-calculate
p =   (np.exp((r-q)*delt) - d)/(u - d)
pu = p*u
pd = (1-p)*d
#print(pu,pd,v)
s_price = [] # list of stock prices tree
v_l = [] # list of u 
o_price_e = [] # list of option prices tree for european option
o_price_a = [] # list of option prices tree for american option
probability_rate = []
for i in range(period+1):
    s_price.append([])
    o_price_e.append([])
    o_price_a.append([])
    v_l.append([])


for i in range(period+1):
    for j in range(i+1):
        v_l[i].append(v**(i-j))

# stock prices and probabilities for all periods
for j in range(period+1):
    for g in range(j+1):
        s_price[j].append(round(s_p*(u**(j-g))*((d)**(g)),10)) # Calculate Stock Price
        
for i in range(int(period/2)+1):
    s_price[i*2][i] = s_p

    
# final return for European / american put option 
# final payoff
for i in range(len(v_l[-1])):
    o_price_e[-1].append(v_l[-1][i] - 1) # Calculate Final Payoff of Option
    o_price_a[-1].append(v_l[-1][i] - 1) # Calculate Final Payoff of Option

# euro put backward
for i in range(period):
    for j in range(period-i-1):
        o_price_e[period-i-1].append((o_price_e[period-i][j]*pd + o_price_e[period-i][j+2]*(pu))*np.exp(-r*delt)) 
    o_price_e[period-i-1].append((o_price_e[period-i][-2]*pd + o_price_e[period-i][-1]*(pu))*np.exp(-r*delt))
#print("1" , o_price_e)      
print("binomial tree model and method in Cheuk and Vorst (1997)")
print("European Put Price : ",round(o_price_e[0][0]*s_p,4))

for i in range(period):
    for j in range(period-i-1):
        o_price_a[period-i-1].append(max(v_l[period-i-1][j]-1,(o_price_a[period-i][j]*pd + o_price_a[period-i][j+2]*(pu))*np.exp(-r*delt)))
    o_price_a[period-i-1].append(max(0,(o_price_a[period-i][-2]*pd + o_price_a[period-i][-1]*(pu))*np.exp(-r*delt)))
#print(o_price_a[0:3])            
print("American Put Price : ",round(o_price_a[0][0]*s_p,4))

binomial tree model and method in Cheuk and Vorst (1997)
European Put Price :  7.61
American Put Price :  7.8086


In [1]:
# Price an American / European lookback put with the binomial tree model
import numpy as np

# Input Info
s_p = 50
q = 0.0
sd1 = 0.4
r = 0.1
t = 0.25
period = 100
smaxt = 70
delt =  t / period
u =  np.exp(sd1 * (delt**0.5))   
d =  1/u         
# pre-calculate
p =   (np.exp((r-q)*delt) - d)/(u - d)
print(u,d,p)
s_price = [] # list of stock prices tree
s_max = [] # list of max stock prices tree 
o_price_e = [] # list of option prices tree for european option
o_price_a = [] # list of option prices tree for american option
probability_rate = []
for i in range(period+1):
    s_price.append([])
    o_price_e.append([])
    o_price_a.append([])
    s_max.append([])

for i in range(period+1):
    for j in range(i+1):
        o_price_e[i].append([])
        o_price_a[i].append([])
        s_max[i].append([])


# stock prices and probabilities for all periods
for j in range(period+1):
    for g in range(j+1):
        s_price[j].append(round(s_p*(u**(j-g))*((d)**(g)),10)) # Calculate Stock Price
        
for i in range(int(period/2)+1):
    s_price[i*2][i] = s_p
# forward tracking methjod
# calculate Monotonic up / down
s_max[0][0] = [smaxt]
for i in range(1,len(s_price)):
    s_max[i][0].append(max(s_price[i][0],smaxt))
    s_max[i][-1].append(smaxt)
# forward tracking methjod test parents
for i in range(1,len(s_price)-1):
    for j in range(i):
        for h in range(len(s_max[i][j])):
            if s_max[i][j][h] >= s_price[i+1][j+1]: # test father
                if s_max[i][j][h] not in s_max[i+1][j+1]: # prevent duplicated insert
                    s_max[i+1][j+1].append(s_max[i][j][h]) # inherit father max
            else:
                if s_price[i+1][j+1] not in s_max[i+1][j+1]:
                    s_max[i+1][j+1].append(s_price[i+1][j+1])
        for g in range(len(s_max[i][j+1])):
            if s_max[i][j+1][g] >= s_price[i+1][j+1]: # test mother
                if s_max[i][j+1][g] not in s_max[i+1][j+1]:
                    s_max[i+1][j+1].append(s_max[i][j+1][g]) # inherit mother max
            else:
                if s_price[i+1][j+1] not in s_max[i+1][j+1]:
                    s_max[i+1][j+1].append(s_price[i+1][j+1])

#print(s_price[-1])
#print(s_max[-1])
    
# final return for European / american put option 
# final payoff
for i in range(len(s_price[-1])):
    for j in range(len(s_max[-1][i])):
        o_price_e[-1][i].append(s_max[-1][i][j] - s_price[-1][i]) # Calculate Final Payoff of Option
        o_price_a[-1][i].append(s_max[-1][i][j] - s_price[-1][i]) # Calculate Final Payoff of Option

#print(s_max)        
# euro put option - backward induction          
for i in range(period):
    for g in range(len(s_max[period-i-1])):
        for h in range(len(s_max[period-i-1][g])):
            if s_max[period-i-1][g][h] in s_max[period-i][g]:
                for f in range(len(s_max[period-i][g])):
                    if s_max[period-i-1][g][h] == s_max[period-i][g][f]:
                        o_u = o_price_e[period-i][g][f]
            else:
                 for v in range(len(s_max[period-i][g])):
                    if s_max[period-i][g][v] == s_price[period-i][g]: # 往上無更大lookback值，故往上值為最大值
                        o_u = o_price_e[period-i][g][v]

            if s_max[period-i-1][g][h] in s_max[period-i][g+1]:
                for v in range(len(s_max[period-i][g+1])):
                    if s_max[period-i-1][g][h] == s_max[period-i][g+1][v]:
                        o_d = o_price_e[period-i][g+1][v]

            value = (o_u*p + o_d*(1-p))*np.exp(-r*delt)
            if value not in o_price_e[period-i-1][g]:
                o_price_e[period-i-1][g].append(value)
print("Option Prices List of European Put ")
print("European Put Price : ",round(o_price_e[0][0][0],4))
#print(o_price_e[-31])

# american put option - backward induction 
for i in range(period):
    for g in range(len(s_max[period-i-1])):
        for h in range(len(s_max[period-i-1][g])):
            if s_max[period-i-1][g][h] in s_max[period-i][g]:
                for f in range(len(s_max[period-i][g])):
                    if s_max[period-i-1][g][h] == s_max[period-i][g][f]:
                        o_u = o_price_a[period-i][g][f]
            else:
                 for v in range(len(s_max[period-i][g])):
                    if s_max[period-i][g][v] == s_price[period-i][g]: # 往上無更大lookback值，故往上值為最大值
                        o_u = o_price_a[period-i][g][v]
                        
            if s_max[period-i-1][g][h] in s_max[period-i][g+1]:
                for v in range(len(s_max[period-i][g+1])):
                    if s_max[period-i-1][g][h] == s_max[period-i][g+1][v]:
                        o_d = o_price_a[period-i][g+1][v]

            value = max((o_u*p + o_d*(1-p))*np.exp(-r*delt),s_max[period-i-1][g][h] - s_price[period-i-1][g])
            o_price_a[period-i-1][g].append(value)
print("Option Prices List of American Put ")
#print(o_price_a[0:3])            
print("American Put Price : ",round(o_price_a[0][0][0],4))

1.0202013400267558 0.9801986733067554 0.5012505312758008
Option Prices List of European Put 
European Put Price :  18.7946
Option Prices List of American Put 
American Put Price :  20


## Monte Carlo

In [2]:
# Calculate Price of European Option by Monte Carlo
from scipy.stats import norm
import math
import numpy as np
size = 10000
# Input Info
t = 0.0
p1 = 50
r1 = 0.1
q1 = 0.0
sd1 = 0.4
smaxt = 70
t1 = 0.25

period = 300
delt = (t1 - t) / period
# call 
vari = (sd1**2)*delt
return_l = []
ans_l = []
s_l = []
for i in range(20):
    return_l = []
    for j in range(size):
        s_l = [p1]
        ran = np.random.standard_normal(period)
        for g in range(period):
            mean = math.log(s_l[-1])+(r1-q1-((sd1**2)/2))*delt
            s_l.append(np.exp(mean+float(ran[g])*(vari**0.5)))    # 求出各路徑node

        return_l.append(max(max(s_l),smaxt) - s_l[-1])
    ans_l.append(np.mean(return_l)*np.exp(-r1*(t1 - t)))
ans_mean = np.mean(ans_l)
ans_std = np.std(ans_l)
#print(ans_l)
#print(s_l)
print("Calculate Price of European Option by Monte Carlo")
print("put price : ",round(ans_mean,4))
print("95% interval of call price : ",ans_mean- 2*ans_std,ans_mean+2*ans_std)

Calculate Price of European Option by Monte Carlo
put price :  18.8155
95% interval of call price :  18.636631346387183 18.99440702187485
